# Lab - working with online data 

In this Lab , you will load , clean and store online flights data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import datetime as dt
%matplotlib inline

In [2]:
import requests
import json


# Load the data

In [3]:
r = requests.get("https://data.gov.il/api/3/action/datastore_search?resource_id=e83f763b-b7d7-479e-b172-ae981ddc6de5&limit=5000")

In [4]:
res = json.loads(r.content)

In [5]:
df = pd.DataFrame(res['result']['records'])

## display the first rows 

In [6]:
df.head(10)

,_id,CHOPER,CHFLTN,CHOPERD,CHSTOL,CHPTOL,CHAORD,CHLOC1,CHLOC1D,CHLOC1TH,CHLOC1T,CHLOC1CH,CHLOCCT,CHTERM,CHCINT,CHCKZN,CHRMINE,CHRMINH
0,1,LY,5109,EL AL ISRAEL AIRLINES,2024-02-27T17:30:00,2024-02-27T17:46:00,D,TBS,TBILISI,טביליסי,TBILISI,גיאורגיה,GEORGIA,3,78-99,D,DEPARTED,המריאה
1,2,AA,387,AMERICAN AIRLINES,2024-02-27T17:30:00,2024-02-27T18:04:00,A,FCO,ROME - FIUMICINO,רומא,ROME,איטליה,ITALY,3,None,None,LANDED,נחתה
2,3,AR,7874,AEROLINEAS ARGENTINAS S.A.,2024-02-27T17:30:00,2024-02-27T18:04:00,A,FCO,ROME - FIUMICINO,רומא,ROME,איטליה,ITALY,3,None,None,LANDED,נחתה
3,4,LY,384,EL AL ISRAEL AIRLINES,2024-02-27T17:30:00,2024-02-27T18:04:00,A,FCO,ROME - FIUMICINO,רומא,ROME,איטליה,ITALY,3,None,None,LANDED,נחתה
4,5,SK,3166,SAS,2024-02-27T17:30:00,2024-02-27T18:04:00,A,FCO,ROME - FIUMICINO,רומא,ROME,איטליה,ITALY,3,None,None,LANDED,נחתה
5,6,TP,8897,AIR PORTUGAL,2024-02-27T17:30:00,2024-02-27T18:04:00,A,FCO,ROME - FIUMICINO,רומא,ROME,איטליה,ITALY,3,None,None,LANDED,נחתה
6,7,LY,571,EL AL ISRAEL AIRLINES,2024-02-27T17:50:00,2024-02-27T18:05:00,D,OTP,BUCHAREST - OTOPENI,בוקרשט,BUCHAREST,רומניה,ROMANIA,3,78-99,D,DEPARTED,המריאה
7,8,6H,045,ISRAIR AIRLINES,2024-02-27T18:00:00,2024-02-27T18:16:00,D,ETM,RAMON,אילת - רמון,RAMON,ישראל,ISRAEL,3,G1-G5,G,DEPARTED,המריאה
8,9,IZ,211,ARKIA ISRAELI AIRLINES,2024-02-27T18:20:00,2024-02-27T18:28:00,D,ATH,ATHENS,אתונה,ATHENS,יוון,GREECE,3,56-60,C,DEPARTED,המריאה
9,10,LY,2523,EL AL ISRAEL AIRLINES,2024-02-27T17:45:00,2024-02-27T18:46:00,D,PRG,PRAGUE-RUZYNE,פראג,PRAGUE,צ'כיה,CZECH REPUBLIC,3,78-99,D,DEPARTED,המריאה


### display the last 10 rows - What's wrong? 

In [7]:
df.tail(10)

# I assume there shuold be many nulls, but there aren't except for 2 specific columns

,_id,CHOPER,CHFLTN,CHOPERD,CHSTOL,CHPTOL,CHAORD,CHLOC1,CHLOC1D,CHLOC1TH,CHLOC1T,CHLOC1CH,CHLOCCT,CHTERM,CHCINT,CHCKZN,CHRMINE,CHRMINH
1013,1014,6H,145,ISRAIR AIRLINES,2024-03-02T16:20:00,2024-03-02T16:20:00,D,GNB,GRENOBLE-SAUNT GEOIRS,גרנובל,GRENOBLE,צרפת,FRANCE,3,None,None,ON TIME,בזמן
1014,1015,BZ,702,BLUE BIRD AIRWAYS,2024-03-02T16:30:00,2024-03-02T16:30:00,D,ATH,ATHENS,אתונה,ATHENS,יוון,GREECE,3,None,None,ON TIME,בזמן
1015,1016,IZ,211,ARKIA ISRAELI AIRLINES,2024-03-02T16:30:00,2024-03-02T16:30:00,D,ATH,ATHENS,אתונה,ATHENS,יוון,GREECE,3,None,None,ON TIME,בזמן
1016,1017,IZ,414,ARKIA ISRAELI AIRLINES,2024-03-02T16:30:00,2024-03-02T16:30:00,A,BUS,BATUMI,בטומי,BATUMI,גיאורגיה,GEORGIA,3,None,None,NOT FINAL,לא סופי
1017,1018,LH,687,LUFTHANSA,2024-03-02T16:45:00,2024-03-02T16:45:00,D,FRA,FRANKFURT,פרנקפורט,FRANKFURT,גרמניה,GERMANY,3,None,None,ON TIME,בזמן
1018,1019,SN,7174,BRUSSELS AIRLINES,2024-03-02T16:45:00,2024-03-02T16:45:00,D,FRA,FRANKFURT,פרנקפורט,FRANKFURT,גרמניה,GERMANY,3,None,None,ON TIME,בזמן
1019,1020,UA,9123,UNITED AIRLINES,2024-03-02T16:45:00,2024-03-02T16:45:00,D,FRA,FRANKFURT,פרנקפורט,FRANKFURT,גרמניה,GERMANY,3,None,None,ON TIME,בזמן
1020,1021,IZ,1843,ARKIA ISRAELI AIRLINES,2024-03-02T16:50:00,2024-03-02T16:50:00,D,ETM,RAMON,אילת - רמון,RAMON,ישראל,ISRAEL,3,None,None,ON TIME,בזמן
1021,1022,LX,252,SWISS INT`L AIRLINES,2024-03-02T17:10:00,2024-03-02T17:10:00,A,ZRH,ZURICH,ציריך,ZURICH,שוויץ,SWITZERLAND,3,None,None,NOT FINAL,לא סופי
1022,1023,A4,5026,AZIMUTH AIRLINES,2024-03-02T17:35:00,2024-03-02T17:35:00,D,AER,SOCHI,סוצ'י,SOCHI,רוסיה,RUSSIAN FEDERATION,3,None,None,ON TIME,בזמן


# clean the data

how many rows contains empty strings for each Column? (use a for loop to display the column name and percentage)

In [8]:
print('Column    Null(%)')
print('-----------------')

for col in df.columns:
    nulls = df[col].isna().sum()
    total = len(df)
    print('{:<8} {:>8}'.format(str(col), np.round(100*nulls/total, 1)))

Column    Null(%)
-----------------
_id           0.0
CHOPER        0.0
CHFLTN        0.0
CHOPERD       0.0
CHSTOL        0.0
CHPTOL        0.0
CHAORD        0.0
CHLOC1        0.0
CHLOC1D       0.0
CHLOC1TH      0.0
CHLOC1T       0.0
CHLOC1CH      0.0
CHLOCCT       0.0
CHTERM        0.0
CHCINT       54.6
CHCKZN       54.6
CHRMINE       0.0
CHRMINH       0.0


drop the empty rows  

In [9]:
for col in df.columns[~df.columns.isin(df[['CHCINT', 'CHCKZN']])]:
    ls = list(df[df[col].isnull()].index)
    df.drop(ls, axis = 0, inplace=True)
df

# Not expecting any rows to be deleted as there are no columns other than 'CHCINT' and 'CHCKZN' with null values

,_id,CHOPER,CHFLTN,CHOPERD,CHSTOL,CHPTOL,CHAORD,CHLOC1,CHLOC1D,CHLOC1TH,CHLOC1T,CHLOC1CH,CHLOCCT,CHTERM,CHCINT,CHCKZN,CHRMINE,CHRMINH
0,1,LY,5109,EL AL ISRAEL AIRLINES,2024-02-27T17:30:00,2024-02-27T17:46:00,D,TBS,TBILISI,טביליסי,TBILISI,גיאורגיה,GEORGIA,3,78-99,D,DEPARTED,המריאה
1,2,AA,387,AMERICAN AIRLINES,2024-02-27T17:30:00,2024-02-27T18:04:00,A,FCO,ROME - FIUMICINO,רומא,ROME,איטליה,ITALY,3,None,None,LANDED,נחתה
2,3,AR,7874,AEROLINEAS ARGENTINAS S.A.,2024-02-27T17:30:00,2024-02-27T18:04:00,A,FCO,ROME - FIUMICINO,רומא,ROME,איטליה,ITALY,3,None,None,LANDED,נחתה
3,4,LY,384,EL AL ISRAEL AIRLINES,2024-02-27T17:30:00,2024-02-27T18:04:00,A,FCO,ROME - FIUMICINO,רומא,ROME,איטליה,ITALY,3,None,None,LANDED,נחתה
4,5,SK,3166,SAS,2024-02-27T17:30:00,2024-02-27T18:04:00,A,FCO,ROME - FIUMICINO,רומא,ROME,איטליה,ITALY,3,None,None,LANDED,נחתה
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,1019,SN,7174,BRUSSELS AIRLINES,2024-03-02T16:45:00,2024-03-02T16:45:00,D,FRA,FRANKFURT,פרנקפורט,FRANKFURT,גרמניה,GERMANY,3,None,None,ON TIME,בזמן
1019,1020,UA,9123,UNITED AIRLINES,2024-03-02T16:45:00,2024-03-02T16:45:00,D,FRA,FRANKFURT,פרנקפורט,FRANKFURT,גרמניה,GERMANY,3,None,None,ON TIME,בזמן
1020,1021,IZ,1843,ARKIA ISRAELI AIRLINES,2024-03-02T16:50:00,2024-03-02T16:50:00,D,ETM,RAMON,אילת - רמון,RAMON,ישראל,ISRAEL,3,None,None,ON TIME,בזמן
1021,1022,LX,252,SWISS INT`L AIRLINES,2024-03-02T17:10:00,2024-03-02T17:10:00,A,ZRH,ZURICH,ציריך,ZURICH,שוויץ,SWITZERLAND,3,None,None,NOT FINAL,לא סופי


format the current date with underscore separation (%d_%m_%y)

In [10]:
today = dt.datetime.now().strftime("%d_%m_%Y")
today

'28_02_2024'

Drop the _id column 

In [11]:
df.drop('_id', axis = 1, inplace = True)
df

,CHOPER,CHFLTN,CHOPERD,CHSTOL,CHPTOL,CHAORD,CHLOC1,CHLOC1D,CHLOC1TH,CHLOC1T,CHLOC1CH,CHLOCCT,CHTERM,CHCINT,CHCKZN,CHRMINE,CHRMINH
0,LY,5109,EL AL ISRAEL AIRLINES,2024-02-27T17:30:00,2024-02-27T17:46:00,D,TBS,TBILISI,טביליסי,TBILISI,גיאורגיה,GEORGIA,3,78-99,D,DEPARTED,המריאה
1,AA,387,AMERICAN AIRLINES,2024-02-27T17:30:00,2024-02-27T18:04:00,A,FCO,ROME - FIUMICINO,רומא,ROME,איטליה,ITALY,3,None,None,LANDED,נחתה
2,AR,7874,AEROLINEAS ARGENTINAS S.A.,2024-02-27T17:30:00,2024-02-27T18:04:00,A,FCO,ROME - FIUMICINO,רומא,ROME,איטליה,ITALY,3,None,None,LANDED,נחתה
3,LY,384,EL AL ISRAEL AIRLINES,2024-02-27T17:30:00,2024-02-27T18:04:00,A,FCO,ROME - FIUMICINO,רומא,ROME,איטליה,ITALY,3,None,None,LANDED,נחתה
4,SK,3166,SAS,2024-02-27T17:30:00,2024-02-27T18:04:00,A,FCO,ROME - FIUMICINO,רומא,ROME,איטליה,ITALY,3,None,None,LANDED,נחתה
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,SN,7174,BRUSSELS AIRLINES,2024-03-02T16:45:00,2024-03-02T16:45:00,D,FRA,FRANKFURT,פרנקפורט,FRANKFURT,גרמניה,GERMANY,3,None,None,ON TIME,בזמן
1019,UA,9123,UNITED AIRLINES,2024-03-02T16:45:00,2024-03-02T16:45:00,D,FRA,FRANKFURT,פרנקפורט,FRANKFURT,גרמניה,GERMANY,3,None,None,ON TIME,בזמן
1020,IZ,1843,ARKIA ISRAELI AIRLINES,2024-03-02T16:50:00,2024-03-02T16:50:00,D,ETM,RAMON,אילת - רמון,RAMON,ישראל,ISRAEL,3,None,None,ON TIME,בזמן
1021,LX,252,SWISS INT`L AIRLINES,2024-03-02T17:10:00,2024-03-02T17:10:00,A,ZRH,ZURICH,ציריך,ZURICH,שוויץ,SWITZERLAND,3,None,None,NOT FINAL,לא סופי


save the raw data (add the current date to the file name) 

In [12]:
df.to_csv('flights_' + today + '.csv')

select the airline code and name without duplicates (CHOPER,CHOPERD)

In [13]:
al1 = df[['CHOPER','CHOPERD']].drop_duplicates()
al1


,CHOPER,CHOPERD
0,LY,EL AL ISRAEL AIRLINES
1,AA,AMERICAN AIRLINES
2,AR,AEROLINEAS ARGENTINAS S.A.
4,SK,SAS
5,TP,AIR PORTUGAL
7,6H,ISRAIR AIRLINES
8,IZ,ARKIA ISRAELI AIRLINES
12,A4,AZIMUTH AIRLINES
16,S7,SIBERIA AIRLINES
18,QF,QANTAS AIRWAYS


Load the file airlines.csv and merge it with a new data and save the file 'airline.csv'

In [14]:
al2 = pd.read_csv('airlines.csv')
airlines = pd.concat([al1,al2]).drop_duplicates()
airlines.to_csv('airlines.csv', index=False)

Drop the Airline name column 

In [15]:
df.drop('CHOPERD', axis = 1, inplace = True)
df

,CHOPER,CHFLTN,CHSTOL,CHPTOL,CHAORD,CHLOC1,CHLOC1D,CHLOC1TH,CHLOC1T,CHLOC1CH,CHLOCCT,CHTERM,CHCINT,CHCKZN,CHRMINE,CHRMINH
0,LY,5109,2024-02-27T17:30:00,2024-02-27T17:46:00,D,TBS,TBILISI,טביליסי,TBILISI,גיאורגיה,GEORGIA,3,78-99,D,DEPARTED,המריאה
1,AA,387,2024-02-27T17:30:00,2024-02-27T18:04:00,A,FCO,ROME - FIUMICINO,רומא,ROME,איטליה,ITALY,3,None,None,LANDED,נחתה
2,AR,7874,2024-02-27T17:30:00,2024-02-27T18:04:00,A,FCO,ROME - FIUMICINO,רומא,ROME,איטליה,ITALY,3,None,None,LANDED,נחתה
3,LY,384,2024-02-27T17:30:00,2024-02-27T18:04:00,A,FCO,ROME - FIUMICINO,רומא,ROME,איטליה,ITALY,3,None,None,LANDED,נחתה
4,SK,3166,2024-02-27T17:30:00,2024-02-27T18:04:00,A,FCO,ROME - FIUMICINO,רומא,ROME,איטליה,ITALY,3,None,None,LANDED,נחתה
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,SN,7174,2024-03-02T16:45:00,2024-03-02T16:45:00,D,FRA,FRANKFURT,פרנקפורט,FRANKFURT,גרמניה,GERMANY,3,None,None,ON TIME,בזמן
1019,UA,9123,2024-03-02T16:45:00,2024-03-02T16:45:00,D,FRA,FRANKFURT,פרנקפורט,FRANKFURT,גרמניה,GERMANY,3,None,None,ON TIME,בזמן
1020,IZ,1843,2024-03-02T16:50:00,2024-03-02T16:50:00,D,ETM,RAMON,אילת - רמון,RAMON,ישראל,ISRAEL,3,None,None,ON TIME,בזמן
1021,LX,252,2024-03-02T17:10:00,2024-03-02T17:10:00,A,ZRH,ZURICH,ציריך,ZURICH,שוויץ,SWITZERLAND,3,None,None,NOT FINAL,לא סופי


Do the same task with airports (CHLOC1,CHLOC1D) and save the file 'airports.csv'

In [16]:
ap1 = df[['CHLOC1','CHLOC1D']]
ap2 = pd.read_csv('airports.csv')
airports = pd.concat([ap1,ap2]).drop_duplicates()
airports.to_csv('airports.csv', index=False)

drop the hebrew columns 'CHLOC1TH','CHLOC1CH','CHRMINH'

In [17]:
df.drop(['CHLOC1TH','CHLOC1CH','CHRMINH'], axis = 1, inplace = True)
df

,CHOPER,CHFLTN,CHSTOL,CHPTOL,CHAORD,CHLOC1,CHLOC1D,CHLOC1T,CHLOCCT,CHTERM,CHCINT,CHCKZN,CHRMINE
0,LY,5109,2024-02-27T17:30:00,2024-02-27T17:46:00,D,TBS,TBILISI,TBILISI,GEORGIA,3,78-99,D,DEPARTED
1,AA,387,2024-02-27T17:30:00,2024-02-27T18:04:00,A,FCO,ROME - FIUMICINO,ROME,ITALY,3,None,None,LANDED
2,AR,7874,2024-02-27T17:30:00,2024-02-27T18:04:00,A,FCO,ROME - FIUMICINO,ROME,ITALY,3,None,None,LANDED
3,LY,384,2024-02-27T17:30:00,2024-02-27T18:04:00,A,FCO,ROME - FIUMICINO,ROME,ITALY,3,None,None,LANDED
4,SK,3166,2024-02-27T17:30:00,2024-02-27T18:04:00,A,FCO,ROME - FIUMICINO,ROME,ITALY,3,None,None,LANDED
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,SN,7174,2024-03-02T16:45:00,2024-03-02T16:45:00,D,FRA,FRANKFURT,FRANKFURT,GERMANY,3,None,None,ON TIME
1019,UA,9123,2024-03-02T16:45:00,2024-03-02T16:45:00,D,FRA,FRANKFURT,FRANKFURT,GERMANY,3,None,None,ON TIME
1020,IZ,1843,2024-03-02T16:50:00,2024-03-02T16:50:00,D,ETM,RAMON,RAMON,ISRAEL,3,None,None,ON TIME
1021,LX,252,2024-03-02T17:10:00,2024-03-02T17:10:00,A,ZRH,ZURICH,ZURICH,SWITZERLAND,3,None,None,NOT FINAL


### rename the columns 

In [18]:
df.rename(columns = {
    'CHOPER': 'airline_code',
    'CHFLTN': 'flight_number',
    'CHSTOL': 'planned_time',
    'CHPTOL': 'actual_time',
    'CHAORD': 'runway',
    'CHLOC1': 'airport_code',
    'CHLOC1D': 'airport_name',
    'CHLOC1T': 'location',
    'CHLOCCT': 'country',	
    'CHTERM': 'terminal',	
    'CHCINT': 'checkin_counters',	
    'CHCKZN': 'checkin_zone',
    'CHRMINE': 'status',
},
inplace=True)

In [19]:
df

,airline_code,flight_number,planned_time,actual_time,runway,airport_code,airport_name,location,country,terminal,checkin_counters,checkin_zone,status
0,LY,5109,2024-02-27T17:30:00,2024-02-27T17:46:00,D,TBS,TBILISI,TBILISI,GEORGIA,3,78-99,D,DEPARTED
1,AA,387,2024-02-27T17:30:00,2024-02-27T18:04:00,A,FCO,ROME - FIUMICINO,ROME,ITALY,3,None,None,LANDED
2,AR,7874,2024-02-27T17:30:00,2024-02-27T18:04:00,A,FCO,ROME - FIUMICINO,ROME,ITALY,3,None,None,LANDED
3,LY,384,2024-02-27T17:30:00,2024-02-27T18:04:00,A,FCO,ROME - FIUMICINO,ROME,ITALY,3,None,None,LANDED
4,SK,3166,2024-02-27T17:30:00,2024-02-27T18:04:00,A,FCO,ROME - FIUMICINO,ROME,ITALY,3,None,None,LANDED
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,SN,7174,2024-03-02T16:45:00,2024-03-02T16:45:00,D,FRA,FRANKFURT,FRANKFURT,GERMANY,3,None,None,ON TIME
1019,UA,9123,2024-03-02T16:45:00,2024-03-02T16:45:00,D,FRA,FRANKFURT,FRANKFURT,GERMANY,3,None,None,ON TIME
1020,IZ,1843,2024-03-02T16:50:00,2024-03-02T16:50:00,D,ETM,RAMON,RAMON,ISRAEL,3,None,None,ON TIME
1021,LX,252,2024-03-02T17:10:00,2024-03-02T17:10:00,A,ZRH,ZURICH,ZURICH,SWITZERLAND,3,None,None,NOT FINAL


Change the PlanTime and ActualTime columns type to np.datetime64

In [20]:
df[['planned_time', 'actual_time']] = df[['planned_time', 'actual_time']].astype('datetime64[s]')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1023 entries, 0 to 1022
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype        
---  ------            --------------  -----        
 0   airline_code      1023 non-null   object       
 1   flight_number     1023 non-null   object       
 2   planned_time      1023 non-null   datetime64[s]
 3   actual_time       1023 non-null   datetime64[s]
 4   runway            1023 non-null   object       
 5   airport_code      1023 non-null   object       
 6   airport_name      1023 non-null   object       
 7   location          1023 non-null   object       
 8   country           1023 non-null   object       
 9   terminal          1023 non-null   int64        
 10  checkin_counters  464 non-null    object       
 11  checkin_zone      464 non-null    object       
 12  status            1023 non-null   object       
dtypes: datetime64[s](2), int64(1), object(10)
memory usage: 104.0+ KB


Save the clean data 

In [21]:
df.to_csv('final.csv', index=False)

update the file final.csv with departed, landed and canceled flights only

In [22]:
new_df = df.query('status in ("DEPARTED", "LANDED", "CANCELLED")')
new_df
new_df.to_csv('final.csv', index=False)

Add a new column for total delay in minutes

In [23]:
df['delay'] = (df['actual_time'] - df['planned_time']).dt.total_seconds()/60
df

,airline_code,flight_number,planned_time,actual_time,runway,airport_code,airport_name,location,country,terminal,checkin_counters,checkin_zone,status,delay
0,LY,5109,2024-02-27 17:30:00,2024-02-27 17:46:00,D,TBS,TBILISI,TBILISI,GEORGIA,3,78-99,D,DEPARTED,16.0
1,AA,387,2024-02-27 17:30:00,2024-02-27 18:04:00,A,FCO,ROME - FIUMICINO,ROME,ITALY,3,None,None,LANDED,34.0
2,AR,7874,2024-02-27 17:30:00,2024-02-27 18:04:00,A,FCO,ROME - FIUMICINO,ROME,ITALY,3,None,None,LANDED,34.0
3,LY,384,2024-02-27 17:30:00,2024-02-27 18:04:00,A,FCO,ROME - FIUMICINO,ROME,ITALY,3,None,None,LANDED,34.0
4,SK,3166,2024-02-27 17:30:00,2024-02-27 18:04:00,A,FCO,ROME - FIUMICINO,ROME,ITALY,3,None,None,LANDED,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,SN,7174,2024-03-02 16:45:00,2024-03-02 16:45:00,D,FRA,FRANKFURT,FRANKFURT,GERMANY,3,None,None,ON TIME,0.0
1019,UA,9123,2024-03-02 16:45:00,2024-03-02 16:45:00,D,FRA,FRANKFURT,FRANKFURT,GERMANY,3,None,None,ON TIME,0.0
1020,IZ,1843,2024-03-02 16:50:00,2024-03-02 16:50:00,D,ETM,RAMON,RAMON,ISRAEL,3,None,None,ON TIME,0.0
1021,LX,252,2024-03-02 17:10:00,2024-03-02 17:10:00,A,ZRH,ZURICH,ZURICH,SWITZERLAND,3,None,None,NOT FINAL,0.0


What is the average delay for departed flights

In [24]:
df.query('status == "DEPARTED"')['delay'].mean()

24.18881118881119

Display total delay distribution 